In [1]:
import os
import os.path
import pandas as pd
import pickle
from PIL import Image, ImageDraw

In [2]:
def readPickle(filename):

    # reload a file to a variable
    with open(filename, 'rb') as file:
        a_dict1 =pickle.load(file)
        
    return a_dict1

In [ ]:
def drawPicture(name, xfFix, yfFix, xfTracing, yfTracing):
    name = name.replace(' ', '(space)').replace('<', '(back)')
    image = Image.open(r'..//img/back.png')
    
    draw = ImageDraw.Draw(image)

    drawLine(draw, xfFix, yfFix, xfTracing, yfTracing)
    drawEllipse(draw, xfFix, yfFix, xfTracing, yfTracing)
        
    image.save(r'..//' + folder + '/' + name + '.png')

In [ ]:
def drawLine():
    draw.line((xf[i-1], yf[i-1], xf[i], yf[i]), fill=(255, 192, 203), width=10)
    
def drawEllipse():
    if len(xf) is not 0: draw.ellipse((xf[0]-10, yf[0]-10, xf[0]+10, yf[0]+10), fill=(255,0,0), outline=None)

In [6]:
dictFix = readPickle('fixationTotalDict.pkl')['509']['1']
dictTracing = readPickle('tracingTotalDict.pkl')['509']['1']
folder = 'result'

#このデータが所持しているsentenceを出す
sentence = []
for index in dictFix.keys():
    sentence.append(index)
    
##################################

#sentenceの数だけ回す    
for index in range(len(sentence)):
    
##########
    
    spacePlace = [] #スペースの位置を保管
    
    
    for space in range(len(dictFix[sentence[index]]['key'])):
        if dictFix[sentence[index]]['key'][space] is ' ' : spacePlace.append(space)
            
    #####
    
    start = 1
    name = 
    for space in range(1, len(spacePlace)):
        
        #空白までの情報を取る
        for pick in range(start, spacePlace[space]):
            xfFix.append(dictFix[sentence[index]]['xf'][pick])
            yfFix.append(dictFix[sentence[index]]['yf'][pick])
            xfTracing.append(dictTracing[sentence[index]]['xf'][pick])
            yfTracing.append(dictTracing[sentence[index]]['yf'][pick])
            
        #####
        
        #画像に描写する
        drawPicture(name, xfFix, yfFix, xfTracing, yfTracing)
                
        #####
        
        start = spacePlace[space]+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
0
1
2
3
4
5
6
7
8
